In [1]:
# chose the current file directory as the working directory
import os 
os.chdir("/teamspace/studios/this_studio/rubikscubesolver")


In [2]:
from tqdm import tqdm

import wandb  # for logging
import time
from dataclasses import dataclass

import jax
import jax.numpy as jnp
import flax.nnx as nnx

import optax

from rubiktransformer.models import RubikTransformer, PolicyModel
import rubiktransformer.dataset as dataset
from rubiktransformer.trainer import train
from rubiktransformer.trainer import reshape_sample

cuda_plugin_extension is not found.


In [3]:
@dataclass
class Config:
    """Configuration class"""
    jax_key: jnp.ndarray = jax.random.PRNGKey(46)
    rngs = nnx.Rngs(45)
    batch_size: int = 128
    lr_1: float = 4e-3
    lr_2: float = 4e-3
    nb_games: int = 128 * 100
    len_seq: int = 20
    nb_step: int = 1000000
    log_every_step: int = 10
    log_eval_every_step: int = 10
    log_policy_reward_every_step: int = 10
    add_data_every_step: int = 500

config = Config()

# init wandb config
user = "forbu14"
project = "RubikTransformer"
display_name = "experiment_" + time.strftime("%Y%m%d-%H%M%S")

wandb.init(entity=user, project=project, name=display_name)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: forbu14. Use `wandb login --relogin` to force relogin


In [4]:
policy = PolicyModel(rngs=config.rngs)
transformer = RubikTransformer(rngs=config.rngs, causal=True)

scheduler = optax.linear_schedule(init_value=0., end_value=1., transition_steps=4000)

# init optimizer
optimizer_optaxworldmodel = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.lion(config.lr_1 / 100.),
   # optax.adamw(config.lr_1/10.),
    optax.scale_by_schedule(scheduler),
)

optimizer_worldmodel = nnx.Optimizer(transformer, optimizer_optaxworldmodel)
optimizer_policy = nnx.Optimizer(policy, optax.adam(config.lr_2))

# metrics
metrics_train = nnx.MultiMetric(
    loss=nnx.metrics.Average("loss"),
    loss_reward=nnx.metrics.Average("loss_reward"),
    loss_cross_entropy=nnx.metrics.Average("loss_cross_entropy"),
)

metrics_eval = nnx.MultiMetric(
    loss_eval=nnx.metrics.Average("loss_eval"),
    loss_reward_eval=nnx.metrics.Average("loss_reward_eval"),
    loss_cross_entropy_eval=nnx.metrics.Average("loss_cross_entropy_eval"),
)


In [5]:
# import pickle

# filename = "state_probainput_vscale3.pickle"

# with open(filename, "rb") as input_file:
#     state = pickle.load(input_file)

# nnx.update(transformer, state)

In [6]:
# gather data from the environment
# init models and optimizers
env, buffer = dataset.init_env_buffer(sample_batch_size=config.batch_size)
env, buffer_eval = dataset.init_env_buffer(sample_batch_size=config.batch_size)


nb_games = config.nb_games
len_seq = config.len_seq

state_first = jnp.zeros((6, 3, 3))
state_next = jnp.zeros((len_seq, 6, 3, 3))
action = jnp.zeros((len_seq, 3))
action_proba = jnp.zeros((len_seq, 9))

# transform state to int8 type
state_first = state_first.astype(jnp.int8)
state_next = state_next.astype(jnp.int8)

# action to int32 type
action = action.astype(jnp.int32)

reward = jnp.zeros((len_seq))

jit_step = jax.jit(env.step)

buffer_list = buffer.init(
    {
        "state_first": state_first,
        "action": action,
        "reward": reward,
        "state_next": state_next,
        "action_pred": action_proba,
    }
)

buffer_list_eval = buffer_eval.init(
    {
        "state_first": state_first,
        "action": action,
        "reward": reward,
        "state_next": state_next,
        "action_pred": action_proba,
    }
)


In [7]:

def step_fn(state, key):
    """
    Simple step function
    We choose a random action
    """

    action = jax.random.randint(
        key=key,
        minval=env.action_spec.minimum,
        maxval=env.action_spec.maximum,
        shape=(3,),
    )

    new_state, timestep = jit_step(state, action)
    timestep.extras["action"] = action
    timestep.extras["action_pred"] = jnp.zeros((9,))

    return new_state, timestep

def run_n_steps(state, key, n):
    random_keys = jax.random.split(key, n)
    state, rollout = jax.lax.scan(step_fn, state, random_keys)

    return rollout

vmap_reset = jax.vmap(jax.jit(env.reset))
vmap_step = jax.vmap(run_n_steps, in_axes=(0, 0, None))

In [8]:
scale_factor_0 = 15.
scale_factor_1 = 10.

def step_fn_proba_setup(state, data):
    """
    Simple step function
    We choose a random action
    """

    key1, key2 = data

    # we choose a probability distribution over the action
    action_proba_0 = jax.random.normal(key1, shape=(6,)) * scale_factor_0
    action_proba_1 = jax.random.normal(key2, shape=(3,)) * scale_factor_1

    # softmax over the action
    action_proba_0 = jax.nn.softmax(action_proba_0)
    action_proba_1 = jax.nn.softmax(action_proba_1)

    # sample the action from the probability distribution
    action_proba_0_value = jax.random.categorical(key1, action_proba_0)
    action_proba_1_value = jax.random.categorical(key2, action_proba_1)

    action = jnp.array([action_proba_0_value, jnp.array(0), action_proba_1_value])

    # concat 
    action_proba = jnp.concatenate([action_proba_0, action_proba_1])

    new_state, timestep = jit_step(state, action)
    timestep.extras["action"] = action
    timestep.extras["action_pred"] = action_proba

    return new_state, timestep

def run_n_steps_proba(state, key, n):
    random_keys = jax.random.split(key, (n, 2))
    state, rollout = jax.lax.scan(step_fn_proba_setup, state, random_keys)

    return rollout

vmap_reset = jax.vmap(jax.jit(env.reset))
vmap_step_proba = jax.vmap(run_n_steps_proba, in_axes=(0, 0, None))

In [9]:
nnx.display(transformer)

In [10]:
key, subkey = jax.random.split(config.jax_key)
config.jax_key = key

buffer, buffer_list = dataset.fast_gathering_data(
    env,
    vmap_reset,
    vmap_step,
    int(config.nb_games / 10),
    config.len_seq,
    buffer,
    buffer_list,
    subkey,
)

In [11]:
key, subkey = jax.random.split(config.jax_key)
config.jax_key = key

buffer, buffer_list = dataset.fast_gathering_data(
    env,
    vmap_reset,
    vmap_step_proba,
    int(config.nb_games / 10),
    config.len_seq,
    buffer,
    buffer_list,
    subkey,
)

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 34.8 seconds.), retrying request


In [12]:

sample = buffer.sample(buffer_list, subkey)
sample = reshape_sample(sample)

In [13]:
sample

TrajectoryBufferSample(experience={'action': Array([[[1.0342622e-03, 9.8763175e-02, 1.9616945e-01, ...,
         9.1598278e-01, 7.9124421e-02, 4.8927194e-03],
        [6.5340154e-04, 7.4344707e-01, 2.9543904e-03, ...,
         8.2277584e-01, 1.3074802e-01, 4.6476152e-02],
        [2.9181805e-01, 3.3277448e-02, 1.3126548e-01, ...,
         1.6328433e-03, 9.9831665e-01, 5.0603030e-05],
        ...,
        [3.3290795e-04, 2.8182656e-04, 1.0129113e-05, ...,
         1.2191311e-01, 4.8528105e-01, 3.9280584e-01],
        [7.0439512e-04, 9.9126726e-01, 8.9772133e-04, ...,
         2.1325910e-02, 7.3878825e-01, 2.3988584e-01],
        [1.1097225e-02, 2.3210780e-03, 8.3933562e-02, ...,
         3.7379863e-04, 9.6265030e-01, 3.6975913e-02]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.00

In [11]:

def loss_fn_transformer(model: RubikTransformer, batch):
    state_logits, reward_value = model(batch["state_first"], batch["action"])

    # reshape state_logits
    # from (batch_size, sequence_length, 324) => (batch_size, sequence_length -1, 54, 6)
    state_logits = state_logits[:, 1:, :]
    state_logits = state_logits.reshape(
        (state_logits.shape[0], state_logits.shape[1], 54, 6)
    )

    reward_value = reward_value[:, 1:]

    loss_crossentropy = optax.softmax_cross_entropy_with_integer_labels(
        logits=state_logits, labels=batch["state_next"]
    ).mean()

    loss_reward = jnp.square(reward_value - batch["reward"]).mean()

    loss = loss_crossentropy + loss_reward

    return loss, (loss_crossentropy, loss_reward)

@nnx.jit
def train_step_transformer(
    model: RubikTransformer, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch
):
    """Train for a single step."""
    grad_fn = nnx.value_and_grad(loss_fn_transformer, has_aux=True)
    (loss, (loss_crossentropy, loss_reward)), grads = grad_fn(model, batch)
    metrics.update(
        loss=loss, loss_reward=loss_reward, loss_cross_entropy=loss_crossentropy
    )
    optimizer.update(grads)


In [12]:
key, subkey = jax.random.split(config.jax_key)
config.jax_key = key

buffer, buffer_list = dataset.fast_gathering_data(
    env,
    vmap_reset,
    vmap_step_proba,
    config.nb_games * 10,
    config.len_seq,
    buffer,
    buffer_list,
    subkey,
)


In [13]:

# transformer model calibration
for idx_step in tqdm(range(config.nb_step)):

    # training for world model
    key, subkey = jax.random.split(config.jax_key)
    config.jax_key = key

    if idx_step % config.add_data_every_step == 0:
        buffer, buffer_list = dataset.fast_gathering_data(
            env,
            vmap_reset,
            vmap_step_proba,
            int(config.nb_games / 10.),
            config.len_seq,
            buffer,
            buffer_list,
            config.jax_key,
        )

    sample = buffer.sample(buffer_list, subkey)
    sample = reshape_sample(sample)

    # we update the policy
    train_step_transformer(
        transformer, optimizer_worldmodel, metrics_train, sample.experience
    )

    if idx_step % config.log_every_step == 0:
        metrics_train_result = metrics_train.compute()
        print(metrics_train_result)

        wandb.log(metrics_train_result, step=idx_step)
        metrics_train.reset()

    if idx_step % config.log_eval_every_step == 0:

        scale_factor = 30.

        key, subkey = jax.random.split(config.jax_key)
        config.jax_key = key
        
        buffer_eval, buffer_list_eval = dataset.fast_gathering_data(
            env,
            vmap_reset,
            vmap_step_proba,
            int(128),
            config.len_seq,
            buffer_eval,
            buffer_list_eval,
            subkey,
        )

        sample = buffer_eval.sample(buffer_list_eval, subkey)
        sample = reshape_sample(sample)

        loss, (loss_crossentropy, loss_reward) = loss_fn_transformer(transformer, sample.experience)

        metrics_eval.update(loss_eval=loss, loss_reward_eval=loss_reward, loss_cross_entropy_eval=loss_crossentropy)
        wandb.log(metrics_eval.compute(), step=idx_step)

        metrics_eval.reset()

        scale_factor = 3.



  2%|▏         | 23350/1000000 [1:12:29<35:43:47,  7.59it/s]

{'loss': Array(0.6715029, dtype=float32), 'loss_reward': Array(0.00503205, dtype=float32), 'loss_cross_entropy': Array(0.6664708, dtype=float32)}


  2%|▏         | 23358/1000000 [1:12:30<36:06:33,  7.51it/s]

{'loss': Array(0.6737969, dtype=float32), 'loss_reward': Array(0.00537699, dtype=float32), 'loss_cross_entropy': Array(0.6684199, dtype=float32)}


  2%|▏         | 23370/1000000 [1:12:32<29:14:52,  9.28it/s]

...

  4%|▍         | 43909/1000000 [2:16:07<31:56:22,  8.32it/s]

{'loss': Array(0.3233163, dtype=float32), 'loss_reward': Array(0.00321406, dtype=float32), 'loss_cross_entropy': Array(0.32010224, dtype=float32)}


  4%|▍         | 43920/1000000 [2:16:09<31:30:03,  8.43it/s]

{'loss': Array(0.3253051, dtype=float32), 'loss_reward': Array(0.00336429, dtype=float32), 'loss_cross_entropy': Array(0.32194078, dtype=float32)}


  4%|▍         | 43922/1000000 [2:16:10<65:55:09,  4.03it/s]

In [12]:
sample

TrajectoryBufferSample(experience={'action': Array([[[1.32556781e-01, 7.96739519e-01, 5.36718592e-02, ...,
         3.91646661e-03, 4.48901858e-03, 9.91594553e-01],
        [3.49070907e-01, 4.57749265e-04, 4.38157976e-01, ...,
         7.23136306e-01, 1.23497941e-01, 1.53365776e-01],
        [6.12441264e-03, 2.50436477e-02, 1.35732419e-03, ...,
         3.82237613e-01, 5.98694921e-01, 1.90675538e-02],
        ...,
        [1.41329234e-04, 2.44877161e-03, 8.43136787e-01, ...,
         2.33344346e-01, 6.42170012e-01, 1.24485560e-01],
        [6.32655225e-04, 1.77795421e-02, 9.65278149e-01, ...,
         1.25269741e-02, 3.21629345e-01, 6.65843725e-01],
        [9.08881542e-04, 1.04175135e-01, 7.50824576e-04, ...,
         9.99683421e-03, 7.89827347e-01, 2.00175866e-01]],

       [[2.03237548e-01, 7.00179100e-01, 3.63819454e-05, ...,
         9.96583939e-01, 2.39940570e-03, 1.01662707e-03],
        [7.63220847e-01, 1.11325733e-01, 3.15520242e-02, ...,
         5.45369804e-01, 4.54322606e-0

In [15]:
sample = buffer_eval.sample(buffer_list_eval, subkey)
sample = reshape_sample(sample)


In [16]:
loss, (loss_crossentropy, loss_reward) = loss_fn_transformer(transformer, sample.experience)


AttributeError: 'RubikTransformer' object has no attribute 'state_mapping'

In [17]:
# save buffer, buffer_list
# in pickle 
import pickle

state_weight = nnx.state(transformer)

In [18]:
state_weight

State({
  'action_mapping': {
    'bias': VariableState(
      type=Param,
      value=Array([ 0.05804873,  0.03530794, -0.06344386,  0.11217742, -0.00204838,
              0.03259815,  0.00209457, -0.00875406, -0.02046519,  0.07442955,
              0.04382639, -0.06812178,  0.0239886 ,  0.03021448,  0.03324478,
             -0.00939911,  0.01842497, -0.02299442,  0.00597318, -0.05957094,
              0.05180154,  0.08517693, -0.0014244 ,  0.00738644, -0.06243268,
              0.14644375, -0.00069646,  0.01227313, -0.04554354, -0.0732253 ,
             -0.0509976 , -0.05631914,  0.03738089, -0.05132922,  0.05668032,
             -0.00109585, -0.15057902, -0.05047007, -0.04745406,  0.04642721,
              0.05568941,  0.05438963, -0.06343702,  0.07053652, -0.0265609 ,
              0.05900841,  0.027164  ,  0.07832118, -0.05330597,  0.05487438,
              0.14665098, -0.08351623, -0.09119233, -0.06400578, -0.0773109 ,
              0.05192997, -0.08234955, -0.04352584, -0.020047

In [19]:
# save state into pickle
with open('state_probainput_vscale4.pickle', 'wb') as handle:
    pickle.dump(state_weight, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
state_weight

State({})

In [13]:
nnx.state(transformer)

State({})

In [14]:

transformer

RubikTransformer()